In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Input
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.utils import to_categorical, plot_model
from keras.datasets import mnist
from keras.layers import concatenate
from keras.models import Model

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

레이블 개수 계산

In [ ]:
num_labels = len(np.unique(y_train))
num_labels

10

원-핫 벡터로 변환

In [ ]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
image_size = x_train.shape[1]
image_size

28

크기 조정, 정규화

In [ ]:
x_train = np.reshape(x_train,[-1, image_size, image_size, 1])
x_test = np.reshape(x_test,[-1, image_size, image_size, 1])
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

신경망 매개변수

In [ ]:
input_shape = (image_size, image_size, 1)
batch_size = 32
kernel_size = 3
dropout = 0.4
n_filters = 32

왼쪽 가지

In [ ]:
left_inputs = Input(shape=input_shape)
x = left_inputs
filters = n_filters
# 3 layers of Conv2D-Dropout-MaxPooling2D
# number of filters doubles after each layer (32-64-128)
for i in range(3):
    x = Conv2D(filters=filters,
               kernel_size=kernel_size,
               padding='same',
               activation='relu')(x)
    x = Dropout(dropout)(x)
    x = MaxPooling2D()(x)
    filters *= 2

오른쪽 가지

In [ ]:
right_inputs = Input(shape=input_shape)
y = right_inputs
filters = n_filters
# 3 layers of Conv2D-Dropout-MaxPooling2D
# number of filters doubles after each layer (32-64-128)
for i in range(3):
    y = Conv2D(filters=filters,
               kernel_size=kernel_size,
               padding='same',
               activation='relu',
               dilation_rate=2)(y)
    y = Dropout(dropout)(y)
    y = MaxPooling2D()(y)
    filters *= 2

가지 병합

In [ ]:
# merge left and right branches outputs
y = concatenate([x, y])
# feature maps to vector before connecting to Dense 
y = Flatten()(y)
y = Dropout(dropout)(y)
outputs = Dense(num_labels, activation='softmax')(y)

In [ ]:
model = Model([left_inputs, right_inputs], outputs)
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 input_12 (InputLayer)          [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 conv2d_26 (Conv2D)             (None, 28, 28, 32)   320         ['input_11[0][0]']               
                                                                                                  
 conv2d_29 (Conv2D)             (None, 28, 28, 32)   320         ['input_12[0][0]']               
                                                                                            

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit([x_train, x_train], y_train, validation_data=([x_test, x_test] ,y_test), epochs=20, batch_size = batch_size)

Epoch 1/20
1875/1875 [==============================] - 283s 151ms/step - loss: 2.3017 - accuracy: 0.1115 - val_loss: 2.3013 - val_accuracy: 0.1135
Epoch 2/20
1875/1875 [==============================] - 279s 149ms/step - loss: 2.3014 - accuracy: 0.1124 - val_loss: 2.3011 - val_accuracy: 0.1135
Epoch 3/20
1875/1875 [==============================] - 278s 148ms/step - loss: 2.3013 - accuracy: 0.1124 - val_loss: 2.3010 - val_accuracy: 0.1135
Epoch 4/20
1875/1875 [==============================] - 279s 149ms/step - loss: 2.3014 - accuracy: 0.1124 - val_loss: 2.3011 - val_accuracy: 0.1135
Epoch 5/20
1875/1875 [==============================] - 279s 149ms/step - loss: 2.3013 - accuracy: 0.1124 - val_loss: 2.3011 - val_accuracy: 0.1135
Epoch 6/20
1875/1875 [==============================] - 285s 152ms/step - loss: 2.3014 - accuracy: 0.1124 - val_loss: 2.3011 - val_accuracy: 0.1135
Epoch 7/20
1875/1875 [==============================] - 279s 149ms/step - loss: 2.3013 - accuracy: 0.1124 - val_

In [ ]:
score = model.evaluate([x_test, x_test], y_test, batch_size=batch_size)
score

313/313 [==============================] - 11s 34ms/step - loss: 2.3012 - accuracy: 0.1135


[2.301151752471924, 0.11349999904632568]